In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import math
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1234)

device

device(type='cuda', index=0)

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform = ToTensor()
)


train_dataloader = DataLoader(training_data, batch_size=1024, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=1024)

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)


In [4]:
learning_rate = 0.01
batch_size = 256
epochs = 50

# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.314399  [    0/60000]
Test Error: 
 Accuracy: 82.2%, Avg loss: 0.501172 

Epoch 2
-------------------------------
loss: 0.491128  [    0/60000]
Test Error: 
 Accuracy: 85.0%, Avg loss: 0.413957 

Epoch 3
-------------------------------
loss: 0.406111  [    0/60000]
Test Error: 
 Accuracy: 86.0%, Avg loss: 0.391032 

Epoch 4
-------------------------------
loss: 0.362574  [    0/60000]
Test Error: 
 Accuracy: 86.2%, Avg loss: 0.381274 

Epoch 5
-------------------------------
loss: 0.322637  [    0/60000]
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.364026 

Epoch 6
-------------------------------
loss: 0.285767  [    0/60000]
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.353167 

Epoch 7
-------------------------------
loss: 0.276089  [    0/60000]
Test Error: 
 Accuracy: 86.7%, Avg loss: 0.370231 

Epoch 8
-------------------------------
loss: 0.298020  [    0/60000]
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.353019 

Epoch 9
----------------